In [4]:
# Dependencies
import pandas as pd
import os
import glob
import numpy as np

path = 'Resources'
fastfood_county = pd.DataFrame()

# get all csv files
csv_files = glob.glob(os.path.join(path, "*.csv"))


# Loop through files to store all csvs in a df
for f in csv_files:
    # read the file into dataframe
    file_load = pd.read_csv(f)
    
    # add column County and populate with filename - .csv
    # filename = f.strip("Resources/")
    # filename = filename.strip(".csv")
    # file_load["County"] = filename
    
    #transpose the data and remove the first column
    file_load = file_load.T
    file_load = file_load.iloc[1: , :]
    
    # add it to the main dataframe, ignoring the first 
    fastfood_county = fastfood_county.append(file_load, ignore_index=True)
        
    # print success
    # print(f"loaded {f}")


In [5]:
fastfood_county.rename(columns={ fastfood_county.columns[0]: "County" }, inplace = True)
fastfood_county.rename(columns={ fastfood_county.columns[1]: "Fast_food_2011" }, inplace = True)
fastfood_county.rename(columns={ fastfood_county.columns[2]: "Fast_food_2016" }, inplace = True)

fastfood_county = fastfood_county[["County","Fast_food_2011","Fast_food_2016"]]

In [6]:
fastfood_county.head()

,County,Fast_food_2011,Fast_food_2016
0,Brown,2,4
1,Franklin,23,23
2,Union,11,16
3,Warren,7,8
4,Moultrie,3,3


In [7]:
fastfood_count_2011_df=fastfood_county[["County","Fast_food_2011"]]
fastfood_count_2011_df["Year"] ="2011"
fastfood_count_2011_df = fastfood_count_2011_df.rename( columns = {"Fast_food_2011":"Restaurant Count"})

fastfood_count_2011_df.head()

,County,Restaurant Count,Year
0,Brown,2,2011
1,Franklin,23,2011
2,Union,11,2011
3,Warren,7,2011
4,Moultrie,3,2011


In [8]:
fastfood_count_2016_df=fastfood_county[["County","Fast_food_2016"]]
fastfood_count_2016_df["Year"] ="2016"
fastfood_count_2016_df = fastfood_count_2016_df.rename( columns = {"Fast_food_2016":"Restaurant Count"})

fastfood_count_2016_df.head()

,County,Restaurant Count,Year
0,Brown,4,2016
1,Franklin,23,2016
2,Union,16,2016
3,Warren,8,2016
4,Moultrie,3,2016


In [9]:
fast_food_11_16_df = pd.concat([fastfood_count_2011_df, fastfood_count_2016_df], ignore_index=True)
fast_food_11_16_df

,County,Restaurant Count,Year
0,Brown,2,2011
1,Franklin,23,2011
2,Union,11,2011
3,Warren,7,2011
4,Moultrie,3,2011
...,...,...,...
199,Washington,9,2016
200,Clay,9,2016
201,Johnson,4,2016
202,Fulton,20,2016


In [10]:
#now bring in FIPS number to prepare to get count by county
IL_Zipcode_county = "IL_Zipcode_countyName.csv"
IL_Zipcode_county_df = pd.read_csv(IL_Zipcode_county)
IL_Zipcode_county_df.head()

,ZIP,FIPS,CountyName,St,Name,Lat,Long,Enc_ZIP,Pops,Area,PopDensity
0,60001,17111,McHenry,IL,60001 Alden,42.421141,-88.616252,60033.0,0,0,0
1,60002,17097,Lake,IL,60002 Antioch,42.474263,-88.103323,NaN,0,0,0
2,60004,17031,Cook,IL,60004 Arlington Heights,42.111424,-87.980372,NaN,0,0,0
3,60005,17031,Cook,IL,60005 Arlington Heights,42.064974,-87.982208,NaN,0,0,0
4,60006,17031,Cook,IL,60006 Arlington Heights,42.088133,-87.993611,60005.0,0,0,0


In [11]:
# discovered some inconsistencies in the county name La Salle in future steps....making a change here for consistency
IL_Zipcode_county_df["county2"] = np.where(IL_Zipcode_county_df["CountyName"] == "La Salle", "LaSalle", IL_Zipcode_county_df["CountyName"])
IL_Zipcode_county_df.head()

,ZIP,FIPS,CountyName,St,Name,Lat,Long,Enc_ZIP,Pops,Area,PopDensity,county2
0,60001,17111,McHenry,IL,60001 Alden,42.421141,-88.616252,60033.0,0,0,0,McHenry
1,60002,17097,Lake,IL,60002 Antioch,42.474263,-88.103323,NaN,0,0,0,Lake
2,60004,17031,Cook,IL,60004 Arlington Heights,42.111424,-87.980372,NaN,0,0,0,Cook
3,60005,17031,Cook,IL,60005 Arlington Heights,42.064974,-87.982208,NaN,0,0,0,Cook
4,60006,17031,Cook,IL,60006 Arlington Heights,42.088133,-87.993611,60005.0,0,0,0,Cook


In [12]:
df_grouped = IL_Zipcode_county_df.groupby("county2")
FIPS_grouped_df=df_grouped.max()

In [13]:
merge_11_16_FIPS_df = pd.merge(fast_food_11_16_df, FIPS_grouped_df, how="left", left_on="County", right_on="county2")
merge_11_16_FIPS_df

,County,Restaurant Count,Year,ZIP,FIPS,CountyName,St,Name,Lat,Long,Enc_ZIP,Pops,Area,PopDensity
0,Brown,2,2011,62378,17009,Brown,IL,62378 Versailles,40.008266,-90.656159,NaN,0,0,0
1,Franklin,23,2011,62999,17055,Franklin,IL,62999 Zeigler,38.092406,-88.742271,62896.0,0,0,0
2,Union,11,2011,62998,17181,Union,IL,62998 Wolf Lake,37.572031,-89.171709,NaN,0,0,0
3,Warren,7,2011,61478,17187,Warren,IL,61478 Smithshire,41.062668,-90.505305,NaN,0,0,0
4,Moultrie,3,2011,61951,17139,Moultrie,IL,61951 Sullivan,39.728904,-88.496654,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Washington,9,2016,62877,17189,Washington,IL,62877 Richview,38.456731,-89.163946,62877.0,0,0,0
200,Clay,9,2016,62899,17025,Clay,IL,62899 Xenia,38.767254,-88.356127,62434.0,0,0,0
201,Johnson,4,2016,62995,17087,Johnson,IL,62995 Vienna,37.581137,-88.760657,62923.0,0,0,0
202,Fulton,20,2016,61563,17057,Fulton,IL,61563 Saint David,40.695424,-90.003804,61531.0,0,0,0


In [14]:
# Export the dataframe to csv
merge_11_16_FIPS_df.to_csv("merge_11_16_FIPS_df.csv")

In [15]:
fast_food_11_16_FIPS_df = merge_11_16_FIPS_df.loc[:,["Restaurant Count", "FIPS","Year"]]
fast_food_11_16_FIPS_df

,Restaurant Count,FIPS,Year
0,2,17009,2011
1,23,17055,2011
2,11,17181,2011
3,7,17187,2011
4,3,17139,2011
...,...,...,...
199,9,17189,2016
200,9,17025,2016
201,4,17087,2016
202,20,17057,2016


In [16]:
# Now bring in the 2021 fast food data, which we had to pull from the yelp API
# Need to clean this data to filter on illinois locations only and eliminate duplicate restaurant id's

Fast_food_21_data = "yelp_fastfood_with_duplicates.csv"

In [17]:
Fast_food_21_df = pd.read_csv(Fast_food_21_data)
Fast_food_21_df.head()

,Unnamed: 0,id,name,Address,zip code,state,lat,lng
0,0,O20eGNVmF9FVQd_3t_lcmg,Taco Bell,"['325 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.414234,-88.613502
1,1,h0anlZoscW80pfXOjU-pug,McDonald's,"['352 S Division', 'Harvard, IL 60033']",60033.0,IL,42.409128,-88.612985
2,2,veyT9UbcbsgPupFhdDUfEw,Dairy Queen,"['605 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.406180,-88.614069
3,3,coQSOoH3npgjUPYvHCZuUA,Subway,"['360 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.408491,-88.612911
4,4,Jtyjr-eht4t5OVyZZj5WRA,Culver's,"['1060 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.405404,-88.613068


In [18]:
# clean the data to filter on state = IL, and eliminate the unnamed column
Fast_food_21_IL_df = Fast_food_21_df.loc[   Fast_food_21_df["state"] == "IL", ["id","name","Address","zip code","state","lat","lng"] ]  
Fast_food_21_IL_df.head()

,id,name,Address,zip code,state,lat,lng
0,O20eGNVmF9FVQd_3t_lcmg,Taco Bell,"['325 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.414234,-88.613502
1,h0anlZoscW80pfXOjU-pug,McDonald's,"['352 S Division', 'Harvard, IL 60033']",60033.0,IL,42.409128,-88.612985
2,veyT9UbcbsgPupFhdDUfEw,Dairy Queen,"['605 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.406180,-88.614069
3,coQSOoH3npgjUPYvHCZuUA,Subway,"['360 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.408491,-88.612911
4,Jtyjr-eht4t5OVyZZj5WRA,Culver's,"['1060 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.405404,-88.613068


In [19]:
# Change the zip code to a string
Fast_food_21_IL_df.loc[:, "zip code"] = Fast_food_21_IL_df["zip code"].astype("str")
Fast_food_21_IL_df["zip code str"] = Fast_food_21_IL_df['zip code'].str[:5]
Fast_food_21_IL_df=Fast_food_21_IL_df.loc[ :, ["id","name","Address","zip code str","state","lat","lng"] ]
Fast_food_21_IL_df = Fast_food_21_IL_df.rename( columns = {"zip code str":"zip code"})
Fast_food_21_IL_df.head()

,id,name,Address,zip code,state,lat,lng
0,O20eGNVmF9FVQd_3t_lcmg,Taco Bell,"['325 S Division St', 'Harvard, IL 60033']",60033,IL,42.414234,-88.613502
1,h0anlZoscW80pfXOjU-pug,McDonald's,"['352 S Division', 'Harvard, IL 60033']",60033,IL,42.409128,-88.612985
2,veyT9UbcbsgPupFhdDUfEw,Dairy Queen,"['605 S Division St', 'Harvard, IL 60033']",60033,IL,42.406180,-88.614069
3,coQSOoH3npgjUPYvHCZuUA,Subway,"['360 S Division St', 'Harvard, IL 60033']",60033,IL,42.408491,-88.612911
4,Jtyjr-eht4t5OVyZZj5WRA,Culver's,"['1060 S Division St', 'Harvard, IL 60033']",60033,IL,42.405404,-88.613068


In [20]:
# check for the number of unique id's using using .nunique()
Fast_food_21_IL_df["id"].nunique()

7424

In [21]:
# need to eliminate duplicate restaurant id's.  Will group by id, and make a dataframe by using the max id

distinct_ids = Fast_food_21_IL_df.groupby("id")
distinct_ids_df = distinct_ids.max()
distinct_ids_df.head()

,name,Address,zip code,state,lat,lng
id,,,,,,
--0OK34lFbyyf4q16yDoGw,Popeyes Louisiana Kitchen,"['45 N Northwest Hwy', 'Palatine, IL 60067']",60067,IL,42.111725,-88.033573
--GvS1gnXSqboHluetk9_Q,McDonald's,"['1158 E Main St', 'Galesburg, IL 61401']",61401,IL,40.947068,-90.352368
--dIVNPnqmx9l0UipYX6oQ,Hardee's,"['539 Woodlawn Rd', 'Lincoln, IL 62656']",62656,IL,40.157408,-89.374600
--fVs_GNeavkOOybb5WNaA,Fire It Up,"['1523 N Aurora Rd', 'Ste 107', 'Naperville, I...",60563,IL,41.781745,-88.183961
--kVRdC7JJ2Ph5UaXQw6aA,Jo Jo's Restaurant,"['1935 W Golf Rd', 'Schaumburg, IL 60194']",60194,IL,42.046957,-88.126650


In [22]:
# sort the data and look for strange zip codes
distinct_ids_df.sort_values("zip code")

,name,Address,zip code,state,lat,lng
id,,,,,,
UiQKEUhRjavKEgfe-xBx4Q,Popeyes Louisiana Kitchen,"['980 Indianapolis Blvd', 'Hammond, IL 46320']",46320,IL,41.695881,-87.518176
Y-ApgB3F9x1d2AkC0RKfig,KFC,"['325 W Il Route 173', 'Antioch, IL 60002']",60002,IL,42.471319,-88.094823
nAz9CVQtD_lHfPrqkSyfpw,McDonald's,"['1150 Main St', 'Antioch, IL 60002']",60002,IL,42.471242,-88.094090
X6TKn8wfaUHo-mt8qHuWAA,Burger King,"['338 W Il Rte 173', 'Antioch, IL 60002']",60002,IL,42.470515,-88.094953
MFBl85WYatZoNNvSg4olbQ,Popeyes Louisiana Kitchen,"['483 E Route 173', 'Antioch, IL 60002']",60002,IL,42.466861,-88.064337
...,...,...,...,...,...,...
wkUc3tzylCQEAyONt-reKw,Ned's Shed,"['101 N 1st St', 'Vienna, IL 62995']",62995,IL,37.414634,-88.893445
RR4wdhO8BXalpFPPE2YLKw,McDonald's,"['707 E Vine St', 'Vienna, IL 62995']",62995,IL,37.414934,-88.874011
7Naxbnb4YBoJHSE5c2NlZQ,Subway,"['104 Redbud Ln', 'Vienna, IL 62995']",62995,IL,37.417305,-88.874104


In [23]:
# eliminate the 46320 zip code (atypical for Illinois) and the nan zip code

distinct_ids_df = distinct_ids_df.loc[(distinct_ids_df["zip code"] != "46320") , :]
distinct_ids_df = distinct_ids_df.loc[(distinct_ids_df["zip code"] != "nan") , :]
distinct_ids_df

,name,Address,zip code,state,lat,lng
id,,,,,,
--0OK34lFbyyf4q16yDoGw,Popeyes Louisiana Kitchen,"['45 N Northwest Hwy', 'Palatine, IL 60067']",60067,IL,42.111725,-88.033573
--GvS1gnXSqboHluetk9_Q,McDonald's,"['1158 E Main St', 'Galesburg, IL 61401']",61401,IL,40.947068,-90.352368
--dIVNPnqmx9l0UipYX6oQ,Hardee's,"['539 Woodlawn Rd', 'Lincoln, IL 62656']",62656,IL,40.157408,-89.374600
--fVs_GNeavkOOybb5WNaA,Fire It Up,"['1523 N Aurora Rd', 'Ste 107', 'Naperville, I...",60563,IL,41.781745,-88.183961
--kVRdC7JJ2Ph5UaXQw6aA,Jo Jo's Restaurant,"['1935 W Golf Rd', 'Schaumburg, IL 60194']",60194,IL,42.046957,-88.126650
...,...,...,...,...,...,...
zwDpnlgp2FUUBdNJlxy1eQ,Burger King,"['2701 N Western Ave', 'Chicago, IL 60647']",60647,IL,41.930619,-87.687632
zwlByG9L7fqFFXr_uyXaXg,Bijou Pub,"['225 S Main St', 'Monmouth, IL 61462']",61462,IL,40.910584,-90.648115
zx2Ms-G5pqXI1_A0JVv6gw,Cilantro Taco Grill - Stone Park,"['1500 N Mannheim Rd', 'Stone Park, IL 60165']",60165,IL,41.897586,-87.884143


In [24]:
# convert ZIP and FIPS to strng for easier merging subsequently
IL_Zipcode_county_df.loc[:, "ZIP"] = IL_Zipcode_county_df["ZIP"].astype("str")
IL_Zipcode_county_df.loc[:, "FIPS"] = IL_Zipcode_county_df["FIPS"].astype("str")
IL_Zipcode_county_df.dtypes

ZIP            object
FIPS           object
CountyName     object
St             object
Name           object
Lat           float64
Long          float64
Enc_ZIP       float64
Pops            int64
Area            int64
PopDensity      int64
county2        object
dtype: object

In [25]:
merge_distinct_ids_FIPs_df = pd.merge(distinct_ids_df,IL_Zipcode_county_df,how="left",left_on="zip code", right_on="ZIP" )
merge_distinct_ids_FIPs_df = merge_distinct_ids_FIPs_df[ ["name","Address","zip code","FIPS","CountyName"]]
merge_distinct_ids_FIPs_df

,name,Address,zip code,FIPS,CountyName
0,Popeyes Louisiana Kitchen,"['45 N Northwest Hwy', 'Palatine, IL 60067']",60067,17031,Cook
1,McDonald's,"['1158 E Main St', 'Galesburg, IL 61401']",61401,17095,Knox
2,Hardee's,"['539 Woodlawn Rd', 'Lincoln, IL 62656']",62656,17107,Logan
3,Fire It Up,"['1523 N Aurora Rd', 'Ste 107', 'Naperville, I...",60563,17043,DuPage
4,Jo Jo's Restaurant,"['1935 W Golf Rd', 'Schaumburg, IL 60194']",60194,17031,Cook
...,...,...,...,...,...
7417,Burger King,"['2701 N Western Ave', 'Chicago, IL 60647']",60647,17031,Cook
7418,Bijou Pub,"['225 S Main St', 'Monmouth, IL 61462']",61462,17187,Warren
7419,Cilantro Taco Grill - Stone Park,"['1500 N Mannheim Rd', 'Stone Park, IL 60165']",60165,17031,Cook
7420,Honey-Jam Cafe,"['120 E Boughton Rd', 'Bolingbrook, IL 60440']",60440,17197,Will


In [26]:
# now need to get a count of restaurants by FIPS

distinct_FIPS_2021 = merge_distinct_ids_FIPs_df.groupby("FIPS")
distinct_count_by_FIPS_2021_df = distinct_FIPS_2021.count()
distinct_count_by_FIPS_2021_df

,name,Address,zip code,CountyName
FIPS,,,,
17001,36,36,36,36
17005,8,8,8,8
17007,21,21,21,21
17009,3,3,3,3
17011,12,12,12,12
...,...,...,...,...
17195,26,26,26,26
17197,366,366,366,366
17199,38,38,38,38


In [27]:
# Want FIPS in a column
distinct_count_by_FIPS_2021_df["FIPS"] = distinct_count_by_FIPS_2021_df.index
distinct_count_by_FIPS_2021_df

,name,Address,zip code,CountyName,FIPS
FIPS,,,,,
17001,36,36,36,36,17001
17005,8,8,8,8,17005
17007,21,21,21,21,17007
17009,3,3,3,3,17009
17011,12,12,12,12,17011
...,...,...,...,...,...
17195,26,26,26,26,17195
17197,366,366,366,366,17197
17199,38,38,38,38,17199


In [28]:
#do more data cleaning
distinct_count_by_FIPS_2021_df = distinct_count_by_FIPS_2021_df[["name","FIPS"]]
fast_food_21_df=distinct_count_by_FIPS_2021_df.rename(columns={"name": "Restaurant Count"})
fast_food_21_df

,Restaurant Count,FIPS
FIPS,,
17001,36,17001
17005,8,17005
17007,21,17007
17009,3,17009
17011,12,17011
...,...,...
17195,26,17195
17197,366,17197
17199,38,17199


In [29]:
# set up the data frame to look like 2011-16 dataframe
fast_food_21_df.reset_index(drop=True)
fast_food_21_df["Year"] ="2021"
fast_food_21_df=fast_food_21_df.reset_index(drop=True)
fast_food_21_df

,Restaurant Count,FIPS,Year
0,36,17001,2021
1,8,17005,2021
2,21,17007,2021
3,3,17009,2021
4,12,17011,2021
...,...,...,...
89,26,17195,2021
90,366,17197,2021
91,38,17199,2021
92,195,17201,2021


In [30]:
# Now combine with 2011 and 2016....again adding the ignore_index = True to reset the index
fast_food_11_16_21_df = pd.concat([fast_food_11_16_FIPS_df, fast_food_21_df], ignore_index=True)
fast_food_11_16_21_df

,Restaurant Count,FIPS,Year
0,2,17009,2011
1,23,17055,2011
2,11,17181,2011
3,7,17187,2011
4,3,17139,2011
...,...,...,...
293,26,17195,2021
294,366,17197,2021
295,38,17199,2021
296,195,17201,2021


In [31]:
# Change the FIPS to a string
fast_food_11_16_21_df.loc[:, "FIPS"] = fast_food_11_16_21_df["FIPS"].astype("str")
fast_food_11_16_21_df["FIPS str"] = fast_food_11_16_21_df['FIPS'].str[:5]
fast_food_11_16_21_df = fast_food_11_16_21_df.loc[ :, ["Restaurant Count","FIPS str","Year"] ]
fast_food_11_16_21_df = fast_food_11_16_21_df.rename( columns = {"FIPS str":"FIPS"})
fast_food_11_16_21_df

,Restaurant Count,FIPS,Year
0,2,17009,2011
1,23,17055,2011
2,11,17181,2011
3,7,17187,2011
4,3,17139,2011
...,...,...,...
293,26,17195,2021
294,366,17197,2021
295,38,17199,2021
296,195,17201,2021


In [32]:
# Export the dataframe to csv
fast_food_11_16_21_df.to_csv("fast_food_11_16_21.csv")